In [1]:
%pwd

'e:\\scaledge_projects\\bird_species_classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'e:\\scaledge_projects\\bird_species_classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_momentum: float
    params_classes: int

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets,models,transforms
import torchvision
# from src.bird_classification.entity.config_entity import PrepareBaseModelConfig
from src.bird_classification.utils import *
from src.bird_classification.utils.common import create_directories
from collections import OrderedDict

src\bird_classification\utils\config.yaml


In [71]:
with open("src/bird_classification/utils/params.yaml", 'r') as file:
        params = yaml.safe_load(file)
        print(params)

with open("src/bird_classification/utils/config.yaml", 'r') as file:
        config = yaml.safe_load(file)
        print(config)

{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 32, 'INCLUDE_TOP': False, 'EPOCHS': 2, 'CLASSES': 10, 'MOMENTUM': 0.9, 'LEARNING_RATE': 0.01}
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/Sibasis555/Face_mask_detection/raw/main/datasets/dummy_bird_species.zip', 'local_data_file': 'artifacts/data_ingestion/dummy_bird_species.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.pth', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.pth'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/trai

In [7]:
class ConfigurationManager: #Config Entity data
    def __init__(self):

        self.config = CONFIG_FILE
        self.params = PARAMS_FILE
    
        create_directories([self.config["artifacts_root"]])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config["prepare_base_model"]
        
        create_directories([config["root_dir"]])
        print(self.params)
        print(config["base_model_path"])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config["root_dir"]),
            base_model_path=Path(config["base_model_path"]),
            updated_base_model_path=Path(config["updated_base_model_path"]),
            params_image_size=self.params["IMAGE_SIZE"],
            params_learning_rate=self.params['LEARNING_RATE'],
            params_include_top=self.params["INCLUDE_TOP"],
            params_momentum=self.params["MOMENTUM"],
            params_classes=self.params["CLASSES"]
        )

        return prepare_base_model_config

In [16]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_model(self):
        self.base_model=models.alexnet(weights=None)
        for param in self.base_model.parameters():
            param.required_grad=False
        print(self.config.base_model_path)
        self.save_model(self.config.base_model_path, self.base_model)

    # @staticmethod
    def model_fc_layer(self, classes):
        self.base_model.fc = nn.Sequential(OrderedDict([
                ('dp1', nn.Dropout(0.2)),
                ('fc1', nn.Linear(9216, classes)),
                ('out', nn.Softmax(dim=1))
        ]))
        return self.base_model

    def update_base_model(self):
        self.full_model = self.model_fc_layer(classes=self.config.params_classes)
        # optimizer=torch.optim.SGD(self.full_model.parameters(),lr=self.config.params_learning_rate,momentum=self.config.params_momentum)
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    @staticmethod
    def save_model(path, model):
        torch.save(model,path)

In [17]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_model()
    prepare_base_model.update_base_model()
    
except Exception as e:
    raise e

[2023-07-19 15:54:18,183: INFO: common: created directory at: artifacts]
[2023-07-19 15:54:18,183: INFO: common: created directory at: artifacts/prepare_base_model]
{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 32, 'INCLUDE_TOP': False, 'EPOCHS': 2, 'CLASSES': 10, 'MOMENTUM': 0.9, 'LEARNING_RATE': 0.01}
artifacts/prepare_base_model/base_model.pth
artifacts\prepare_base_model\base_model.pth
hii artifacts\prepare_base_model\base_model.pth
hii artifacts\prepare_base_model\base_model_updated.pth
